## Introduction

This is an example to derive SMIRNOFF typed atom-centerd polarizabilities from quantum mechanically calculated electrostatics potentials with `factorpol` package.

A global optimization is built by concatenating Individual objective function find polarizabilities for the training set.

## Dependencies

- ray
- openff-toolkit
- openff-recharge*
- sqlalchemy
- openeye-toolkits
- scipy

In [1]:
import os
import pickle
from glob import glob

import numpy as np
import pandas as pd
from openff.toolkit import ForceField, Molecule
from pkg_resources import resource_filename

from factorpol.alphas_training import (AlphaData, AlphasTrainer, AlphaWorker,
                                       optimize_alphas)
from factorpol.charge_training import ChargeTrainer
from factorpol.utilities import (calc_rrms, flatten_a_list, Polarizability,
                                 StorageHandler)

cwd = os.getcwd()

/home/wwilla/mambaforge/envs/factorpol/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/wwilla/mambaforge/envs/factorpol/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/wwilla/mambaforge/envs/factorpol/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/wwilla/mambaforge/envs/factorpol/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


## Prepare dataset

Curate QM ESPs data generated in `00-generate-qm-reference.ipynb`.

`off_examples.offxml` is an example OpenForceField style [ForceField file](https://github.com/openforcefield/openff-forcefields/tree/main/openforcefields/offxml). We use the `<vdW>` handeler to label atoms with SMIRNOFF patterns and assign electrostatics parameters until we have a new handeler for polarizability.


In [2]:
off_forcefield = ForceField(resource_filename(
    "factorpol", os.path.join("data", "off_examples.offxml")
))

# Initialize a polarizability 
alphas0 = Polarizability()

Curate QM data and prepare ray workers to optimize polarizabilities

In [3]:
data = AlphaData(
    database_name="factorpol_examples",
    dataset=["CO", "C=C"],
    off_forcefield=off_forcefield,
    polarizability=alphas0,
    num_cpus=2,
)

2023-04-12 23:41:21,086	INFO worker.py:1553 -- Started a local Ray instance.
(pid=547600) /home/wwilla/mambaforge/envs/factorpol/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
(pid=547600)   setattr(self, word, getattr(machar, word).flat[0])
(pid=547600) /home/wwilla/mambaforge/envs/factorpol/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
(pid=547600)   return self._float_to_str(self.smallest_subnormal)
(pid=547600) /home/wwilla/mambaforge/envs/factorpol/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
(pid=547600)   setattr(self, word, getattr(machar, word).flat[0])
(pid=547600) /home/wwilla/mambaforge/envs/factorpol/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The v

Because we have 2 molecules in the training set, each of them has two sets of QM ESPs, which means 1 workers per molecule and a totoal of 2 workers.

In [4]:
print(f"Number of data in training set:\t {len(data.workers)}")

Number of data in training set:	 2


## Optimization

In [5]:
atrain = AlphasTrainer(
    workers=data.workers,
    prior=alphas0,
    working_directory=os.path.join(cwd, "data_alphas_2"),
)

Path exists, deleting


In [6]:
ret = optimize_alphas(worker_list=atrain.workers, solved=True, num_cpus=8)

2023-04-12 23:41:27,255	INFO worker.py:1553 -- Started a local Ray instance.
(pid=549858) /home/wwilla/mambaforge/envs/factorpol/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
(pid=549858)   setattr(self, word, getattr(machar, word).flat[0])
(pid=549858) /home/wwilla/mambaforge/envs/factorpol/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
(pid=549858)   return self._float_to_str(self.smallest_subnormal)
(pid=549858) /home/wwilla/mambaforge/envs/factorpol/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
(pid=549858)   setattr(self, word, getattr(machar, word).flat[0])
(pid=549858) /home/wwilla/mambaforge/envs/factorpol/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The v

## Results

In [7]:
ret[0]

{'[#6:1]': 2.107914309852468 <Unit('angstrom ** 3')>,
 '[#8:1]': 0.7012858554056591 <Unit('angstrom ** 3')>,
 '[#1:1]': 0.014485995610544271 <Unit('angstrom ** 3')>}

In [8]:
#objective
ret[-1]

8.730940687229713e-06